# VGG-like model for the CIFAR-10 dataset

In this notebook, I present a model able to reach 90 % accuracy on test set by using data augmentation and using an architecture inspired by the VGG model.

- Karen Simonyan, Andrew Zisserman, 2015, Very Deep Convolutional Networks for Large-Scale Image Recognition

In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint

import time

/home/arthur/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Data Preparation

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('X shape :', x_train.shape)
print(len(x_train), 'train samples')
print(len(x_test), 'test samples')

X shape : (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
x_train_mean = np.mean(x_train, axis=0)
x_train_std = np.std(x_train, axis=0)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_train_mean)/x_train_std

n_y = 10
y_train = keras.utils.to_categorical(y_train, n_y)
y_test = keras.utils.to_categorical(y_test, n_y)

# Model

In [4]:
input_shape = x_train.shape[1:]
activation = 'relu'
n_l0 = 64

model = Sequential()

model.add(Conv2D(n_l0, (3,3), padding='same', activation=activation, input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(n_l0, (3,3), padding='same', activation=activation))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides=2))
model.add(Dropout(0.5))

model.add(Conv2D(n_l0*2, (3,3), padding='valid', activation=activation))
model.add(BatchNormalization())
model.add(Conv2D(n_l0*2, (3,3), padding='valid', activation=activation))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides=2))
model.add(Dropout(0.5))

model.add(Conv2D(n_l0*4, (3,3), padding='valid', activation=activation))
model.add(BatchNormalization())
model.add(Conv2D(n_l0*4, (3,3), padding='valid', activation=activation))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2), strides=2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256, activation=activation))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_y, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       73856     
__________

# Model training

In [5]:
optimizer = keras.optimizers.adam(lr=0.001)
model.compile(optimizer, keras.losses.categorical_crossentropy, ['accuracy'])

shift = 4/32
generator = ImageDataGenerator(rotation_range=10, width_shift_range=shift, height_shift_range=shift, 
                               horizontal_flip=True)

batch_size = 64
n_steps = x_train.shape[0]//batch_size

save_path = './Model_trained/VGG_2_model_cifar10.h5'
ckeckpoint = ModelCheckpoint(save_path, save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=1)

t0 = time.time()
model.fit_generator(generator.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=n_steps, 
                    epochs=200, validation_data=(x_test, y_test), callbacks=[ckeckpoint, early_stopping])
print('Total training time : %.3f s' %(time.time()-t0))

Epoch 1/200
781/781 [==============================] - 27s 34ms/step - loss: 1.9706 - acc: 0.3299 - val_loss: 1.4087 - val_acc: 0.4941
Epoch 2/200
781/781 [==============================] - 24s 31ms/step - loss: 1.3726 - acc: 0.5030 - val_loss: 1.2141 - val_acc: 0.5902
Epoch 3/200
781/781 [==============================] - 24s 31ms/step - loss: 1.1535 - acc: 0.5913 - val_loss: 1.2079 - val_acc: 0.6051
Epoch 4/200
781/781 [==============================] - 24s 31ms/step - loss: 1.0149 - acc: 0.6446 - val_loss: 0.8662 - val_acc: 0.7027
Epoch 5/200
781/781 [==============================] - 24s 31ms/step - loss: 0.9213 - acc: 0.6799 - val_loss: 0.8226 - val_acc: 0.7155
Epoch 6/200
781/781 [==============================] - 24s 31ms/step - loss: 0.8634 - acc: 0.7013 - val_loss: 0.7509 - val_acc: 0.7426
Epoch 7/200
781/781 [==============================] - 24s 31ms/step - loss: 0.8160 - acc: 0.7204 - val_loss: 0.7602 - val_acc: 0.7392
Epoch 8/200
781/781 [==============================] - 

781/781 [==============================] - 24s 30ms/step - loss: 0.3875 - acc: 0.8690 - val_loss: 0.3994 - val_acc: 0.8655
Epoch 62/200
781/781 [==============================] - 24s 30ms/step - loss: 0.3883 - acc: 0.8674 - val_loss: 0.3789 - val_acc: 0.8733
Epoch 63/200
781/781 [==============================] - 24s 30ms/step - loss: 0.3813 - acc: 0.8702 - val_loss: 0.3918 - val_acc: 0.8693
Epoch 64/200
781/781 [==============================] - 24s 31ms/step - loss: 0.3766 - acc: 0.8714 - val_loss: 0.3692 - val_acc: 0.8762
Epoch 65/200
781/781 [==============================] - 24s 30ms/step - loss: 0.3786 - acc: 0.8703 - val_loss: 0.3623 - val_acc: 0.8755
Epoch 66/200
781/781 [==============================] - 24s 30ms/step - loss: 0.3759 - acc: 0.8715 - val_loss: 0.4113 - val_acc: 0.8660
Epoch 67/200
781/781 [==============================] - 24s 30ms/step - loss: 0.3751 - acc: 0.8712 - val_loss: 0.3520 - val_acc: 0.8824
Epoch 68/200
781/781 [==============================] - 24s 3

# Final evaluation

In [6]:
best_model = keras.models.load_model(save_path)
print('Best model test accuracy :', best_model.evaluate(x_test, y_test, batch_size=64)[1])

10000/10000 [==============================] - 2s 215us/step
Best model test accuracy : 0.8925
